<a href="https://colab.research.google.com/github/terezablahova/MyRepository/blob/main/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import pandas as pd
nltk.__version__


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'3.8.1'

## Extracting general gender words as a separate column


In [4]:
female_words = ["she", "her", "hers", "herself", "woman", "girl", "lady", "women", "girls", "ladies", "girlfriend", "wife", "mother", "mum", "sister", "daughter", "grandmother", "grandma", "aunt",]
male_words = ["he", "him", "his", "himself", "man", "boy", "gentleman", "men", "boys", "gentlemen", "boyfriend", "husband", "father", "dad", "brother", "son", "grandfather", "grandpa", "uncle"]

In [5]:
def add_gender_columns(df, female_words, male_words):
    """
    Adds two columns to the dataframe: 'female_words' and 'male_words'.
    These columns contain words from the respective lists that are found in the 'input_sequence' column.
    """
    def extract_gender_words(text, word_list):
        # Extract words from the input_sequence that match the gender lists
        tokens = word_tokenize(text.lower())
        return [token for token in tokens if token in word_list]
    # Add the new columns to the dataframe
    df["female_words"] = df["input_sequence"].apply(lambda x: extract_gender_words(x, female_words))
    df["male_words"] = df["input_sequence"].apply(lambda x: extract_gender_words(x, male_words))

    return df


In [6]:
def add_gender_columns(df, female_words, male_words):
    """
    Adds two columns to the dataframe: 'female_words' and 'male_words'.
    These columns contain words from the respective lists that are found in the 'input_sequence' column.
    """
    def extract_gender_words(text, word_list):
        # Extract words from the input_sequence that match the gender lists
        tokens = word_tokenize(text.lower())
        return [token for token in tokens if token in word_list]
    # Add the new columns to the dataframe
    df["female_words"] = df["input_sequence"].apply(lambda x: extract_gender_words(x, female_words))
    df["male_words"] = df["input_sequence"].apply(lambda x: extract_gender_words(x, male_words))

    return df



## Freeform dataset

In [7]:
freeform_train = pd.read_csv("/content/drive/My Drive/_THESIS/Commonsense Norm Bank/data/freeform/validation.freeform.tsv", sep="\t")

In [8]:
freeform_train.head()

,Unnamed: 0,input_sequence,class_label,text_label,input_type,pattern,source
0,0,Are you allowed to let things stop you from do...,-1,it's not okay,action_question,Are you allowed to {action}?,social_chem
1,1,Is letting people know you have them in your t...,1,it's kind,action_situation_question,"Is {action} correct, when {situation}?",social_chem
2,2,Given i fell in love with my boyfriend of 3.5 ...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem
3,3,Is expecting the things in your phone to be pr...,0,It's normal,action_situation_question,"Is {action} moral, if {situation}?",social_chem
4,4,sharing embarrassing stories about your signif...,-1,You shouldn't,action,NaN,social_chem


In [9]:
len(freeform_train)

97018

In [10]:
freeform_train.source.value_counts()

,count
source,
social_chem,80800
moral_stories,12000
ethics_commonsense_morality,4218


### NER

I will duplicate the samples in two parts - first I will extract all names and use a dictionary to map the name to a gender. To create duplicates of these sample with the oposite gender, I will extract general nouns and pronouns, such as "he", "she", "man", "woman" and replicate all sample containing these by switching the target noun or pronoun to the corresponding one of the opposite gender. Secondly, I will replace all male names with "Jane" and all female names "John" to create a counterpart to each existing sequence.

In [11]:
#using NER to map names
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_names (text):
  doc = nlp(text)
  names = []
  for ent in doc.ents:
    if ent.label_ == "PERSON":
      names.append(ent.text.lower())
  return names


In [12]:
#this code takes quite some time to execute so the df with extracted names is loaded below
#freeform_train["names"] = freeform_train["input_sequence"].apply(extract_names)

In [13]:
#freeform_train.to_csv("/content/drive/My Drive/_THESIS/Commonsense Norm Bank/data/freeform/train.freeform_with_names.csv")

In [14]:
df = pd.read_csv("/content/drive/My Drive/_THESIS/Commonsense Norm Bank/data/freeform/train.freeform_with_names.csv")

### Other Nouns + Pronouns

In [15]:
df_names_and_genders = add_gender_columns(df, female_words, male_words)

In [16]:
f_freeform = df_names_and_genders[df_names_and_genders["female_words"].apply(len) > 0]

In [17]:
print("len:", len(f_freeform))


len: 17376


In [18]:
f_freeform.class_label.value_counts()

,count
class_label,
-1,6628
0,5450
1,5298


In [19]:
m_freeform = df_names_and_genders[df_names_and_genders["male_words"].apply(len) > 0]

In [20]:
print("len:", len(m_freeform))


len: 19442


In [21]:
m_freeform.class_label.value_counts()

,count
class_label,
-1,7323
1,6738
0,5381


## Duplication of samples for Gender

In [22]:
#we will only retain samples that contain at least on name or gendered word

df = df_names_and_genders[
    (df_names_and_genders["female_words"].apply(len) > 0)|(df_names_and_genders["male_words"].apply(len) > 0)| (df["names"].apply(len) > 0)
]

df.head()

,Unnamed: 0.1,Unnamed: 0,input_sequence,class_label,text_label,input_type,pattern,source,names,female_words,male_words
0,0,0,Are you allowed to let things stop you from do...,-1,it's not okay,action_question,Are you allowed to {action}?,social_chem,[],[],[]
1,1,1,Is letting people know you have them in your t...,1,it's kind,action_situation_question,"Is {action} correct, when {situation}?",social_chem,[],[],[]
2,2,2,Given i fell in love with my boyfriend of 3.5 ...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem,[],[],[boyfriend]
3,3,3,Is expecting the things in your phone to be pr...,0,It's normal,action_situation_question,"Is {action} moral, if {situation}?",social_chem,[],[],[]
4,4,4,sharing embarrassing stories about your signif...,-1,You shouldn't,action,NaN,social_chem,[],[],[]


In [23]:
df = df.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

In [24]:
df.to_csv("/content/drive/My Drive/_THESIS/Commonsense Norm Bank/preprocessed_df.csv")


In [25]:
df = pd.read_csv("/content/drive/My Drive/_THESIS/Commonsense Norm Bank/preprocessed_df.csv")

## Functions to replace gender

Replacing pronouns and general nouns


*   If there is no intersection between the two gender lists and tokens of input_sequence, the sample is droped from the df (to avoid duplicates since I will concatenate df and new_df at the end of the function)
*   Adding a column "gender_pair" to keep track of the corresponding pairs of sequences.

  *   If the row is duplicated with replaced gendered word, the value for gender_pair will be index + "B".
  *   The corresponding (original row) will have the value of index + "A".



In [26]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize

from tqdm import tqdm

# Sample word lists
female_words = ["she", "her", "hers", "herself", "woman", "girl", "lady", "women", "girls", "ladies", "girlfriend", "wife", "mother", "mum", "sister", "daughter", "grandmother", "grandma", "aunt",]
male_words = ["he", "him", "his", "himself", "man", "boy", "gentleman", "men", "boys", "gentlemen", "boyfriend", "husband", "father", "dad", "brother", "son", "grandfather", "grandpa", "uncle"]

def intersection(lst1, lst2, lst3):
    if len(list(set(lst1) & set(lst2))) > 0 or len(list(set(lst1) & set(lst3))) > 0:
      return True
    else :
      return False


def replace_gender(df, column, word_list1, word_list2):
    # Add new columns if they don't exist
    df["gender_pair"] = pd.Series(dtype='str')
    df = df.reset_index(drop=True)
    new_df = df.copy()

    for i in tqdm(range(0, len(df))):
        # Tokenize the input string
        tokens = word_tokenize(df[column][i].lower())

        # Skip rows where no intersection exists between token lists
        if intersection(tokens, word_list1, word_list2) == False:
            new_df.drop(i, inplace=True)
        else:
            # Initialize the string to be the current value in the DataFrame
            string = new_df.loc[i, column]

            # Iterate through each token and replace as necessary
            for token in tokens:
                if token in word_list1:
                    x = word_list1.index(token)
                    # Update string dynamically
                    string = re.sub(r"\b" + token + r"\b", word_list2[x], string)

                elif token in word_list2:
                    x = word_list2.index(token)
                    # Update string dynamically
                    string = re.sub(r"\b" + token + r"\b", word_list1[x], string)

            # Update the DataFrame with the modified string and "gender_pair"
            new_df.at[i, column] = string
            new_df.at[i, "gender_pair"] = str(i) + "B"
            df.at[i, "gender_pair"] = str(i) + "A"

    # Combine original and modified DataFrame
    new_df = pd.concat([new_df, df]).reset_index(drop=True)
    new_df = new_df.fillna(0)
    return new_df


Replacing names - using a python dictionary that maps names to gender from [this source](https://github.com/Bemmu/gender-from-name/blob/master/gender.py). Instead of creating corresponding name-pairs. I will replace any male name with Jane and any female name with John.
Note: This is not the best approach in terms of cultural diversity, but I will focus only on gender for now and then see what else I can encompass in my research.

In [27]:
import json

path = "/content/drive/My Drive/_THESIS/Commonsense Norm Bank/gender.py"

my_locals = {}
with open(path) as file:
  exec(file.read(), my_locals)

gender_dict = my_locals["gender"]

In [28]:
#lowercase dictionary keys so that the function below can find matchs
gender_dict = {k.lower(): v for k, v in gender_dict.items()}

In [29]:
gender_dict["jessica"]

'female'

In [30]:
len(gender_dict) #wow

5177

We will conduct a similar operation with the names and replace all female names by "John" and all female names by "Jane". It is also necessary to check if the input has already been duplicated via the replace_gender() function to avoid duplicating what was already duplicated.

In [31]:
import re
import pandas as pd

# Function to determine pair status (A or B)
def pair_status(i, column, df):
    label = df[column].iloc[i]
    if re.search("B", str(label)):
        return "B"
    elif re.search("A", str(label)):
        return "A"

def str_to_list(string):
  if type(string) != list:
    return eval(string)
  else:
    return string

def replace_name(df, target_column, name_column, dictionary):

    df["name_gender_pair"] = pd.Series(dtype="str")

    unclear_names = []
    new_rows = pd.DataFrame(columns=df.columns)

    for i in tqdm(range(len(df))):
        names = df.loc[i, name_column]
        names = str_to_list(names) #ensuring names are interpreted as a list
        if len(names) == 0:
            df.at[i, "name_gender_pair"] = 0
            continue

        string = df.loc[i, target_column].lower()
        pair_status_label = pair_status(i, "gender_pair", df)

        if pair_status_label == "B":  # Modify the B row
            for name in names:
                name = name.lower()
                gender = dictionary.get(name)
                if gender == "female":
                    string = re.sub(name, "John", string)
                elif gender == "male":
                    string = re.sub(name, "Jane", string)
                else:
                  unclear_names.append(name)
            df.at[i, target_column] = string
            df.at[i, "name_gender_pair"] = str(i) + "B"

            # Locate the original A pair and update its label
            pair_b = df.loc[i, "gender_pair"]
            pair_a = re.sub("B", "A", pair_b)
            original_index = df.index[df["gender_pair"] == pair_a]
            df.at[original_index[0], "name_gender_pair"] = str(i) + "A"


        elif pair_status_label == "A":  # original sample will always remain untransformed
            continue

        else:  # If gender_pair is not set, create a new B pair
            for name in names:
                name = name.lower()
                gender = dictionary.get(name)
                if gender == "female":
                    string = re.sub(name, "John", string)
                elif gender == "male":
                    string = re.sub(name, "Jane", string)
                else:
                  unclear_names.append(name)
            new_row = df.loc[i].copy()
            new_row[target_column] = string
            df.at[i, "name_gender_pair"] = str(i) + "A"
            new_row["name_gender_pair"]= str(i) + "B"
            new_rows = new_rows._append(new_row)


    result_df = pd.concat([df, new_rows]).reset_index(drop=True)

    return result_df, unclear_names


In [32]:
test_df = df[:10].reset_index()

In [33]:
test_df. head(3)

,index,Unnamed: 0,input_sequence,class_label,text_label,input_type,pattern,source,names,female_words,male_words
0,0,0,Are you allowed to let things stop you from do...,-1,it's not okay,action_question,Are you allowed to {action}?,social_chem,[],[],[]
1,1,1,Is letting people know you have them in your t...,1,it's kind,action_situation_question,"Is {action} correct, when {situation}?",social_chem,[],[],[]
2,2,2,Given i fell in love with my boyfriend of 3.5 ...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem,[],[],['boyfriend']


In [34]:
new_test_df = replace_gender(test_df, "input_sequence", female_words, male_words)

100%|██████████| 10/10 [00:00<00:00, 1248.45it/s]


In [35]:
new_test_df

,index,Unnamed: 0,input_sequence,class_label,text_label,input_type,pattern,source,names,female_words,male_words,gender_pair
0,2,2,Given i fell in love with my girlfriend of 3.5...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem,[],[],['boyfriend'],2B
1,6,6,"If not liking my brother in law and niece, is ...",-1,it's wrong,action_situation_question,"If {situation}, is {action} acceptable?",social_chem,[],['sister'],[],6B
2,7,7,Given chris hid hers friends keys in the couch...,-1,it's wrong,action_situation_question,"Given {situation}, is {action} acceptable?",social_chem,['chris hid'],[],['his'],7B
3,9,9,asking your partner to rub your feet when you ...,0,It's ok,action_situation,"{action}, if {situation}",social_chem,[],['wife'],[],9B
4,0,0,Are you allowed to let things stop you from do...,-1,it's not okay,action_question,Are you allowed to {action}?,social_chem,[],[],[],0
5,1,1,Is letting people know you have them in your t...,1,it's kind,action_situation_question,"Is {action} correct, when {situation}?",social_chem,[],[],[],0
6,2,2,Given i fell in love with my boyfriend of 3.5 ...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem,[],[],['boyfriend'],2A
7,3,3,Is expecting the things in your phone to be pr...,0,It's normal,action_situation_question,"Is {action} moral, if {situation}?",social_chem,[],[],[],0
8,4,4,sharing embarrassing stories about your signif...,-1,You shouldn't,action,0,social_chem,[],[],[],0
9,5,5,Is Jane calling the police on the person not w...,-1,it is not acceptable,action_situation_intention_question,Is {action} acceptable from a moral perspectiv...,moral_stories,"['Jane', 'Jane', 'Jane']",[],[],0


In [36]:
new_test_df, name_list = replace_name(new_test_df, "input_sequence", "names", gender_dict)

100%|██████████| 14/14 [00:00<00:00, 2357.20it/s]


Seems like in this particular case, there was a problem either with NER or with tokenization. I will try to fix it below when applying these functions to both datasets and probably encountering more issues with the names.

In [37]:
name_list

['chris hid']

In [38]:
new_test_df

,index,Unnamed: 0,input_sequence,class_label,text_label,input_type,pattern,source,names,female_words,male_words,gender_pair,name_gender_pair
0,2,2,Given i fell in love with my girlfriend of 3.5...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem,[],[],['boyfriend'],2B,0
1,6,6,"If not liking my brother in law and niece, is ...",-1,it's wrong,action_situation_question,"If {situation}, is {action} acceptable?",social_chem,[],['sister'],[],6B,0
2,7,7,given chris hid hers friends keys in the couch...,-1,it's wrong,action_situation_question,"Given {situation}, is {action} acceptable?",social_chem,['chris hid'],[],['his'],7B,2B
3,9,9,asking your partner to rub your feet when you ...,0,It's ok,action_situation,"{action}, if {situation}",social_chem,[],['wife'],[],9B,0
4,0,0,Are you allowed to let things stop you from do...,-1,it's not okay,action_question,Are you allowed to {action}?,social_chem,[],[],[],0,0
5,1,1,Is letting people know you have them in your t...,1,it's kind,action_situation_question,"Is {action} correct, when {situation}?",social_chem,[],[],[],0,0
6,2,2,Given i fell in love with my boyfriend of 3.5 ...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem,[],[],['boyfriend'],2A,0
7,3,3,Is expecting the things in your phone to be pr...,0,It's normal,action_situation_question,"Is {action} moral, if {situation}?",social_chem,[],[],[],0,0
8,4,4,sharing embarrassing stories about your signif...,-1,You shouldn't,action,0,social_chem,[],[],[],0,0
9,5,5,Is Jane calling the police on the person not w...,-1,it is not acceptable,action_situation_intention_question,Is {action} acceptable from a moral perspectiv...,moral_stories,"['Jane', 'Jane', 'Jane']",[],[],0,9A


## Duplicating relevant samples in the whole dataset

In [39]:
print(len(df))
df.head()

97018


,Unnamed: 0,input_sequence,class_label,text_label,input_type,pattern,source,names,female_words,male_words
0,0,Are you allowed to let things stop you from do...,-1,it's not okay,action_question,Are you allowed to {action}?,social_chem,[],[],[]
1,1,Is letting people know you have them in your t...,1,it's kind,action_situation_question,"Is {action} correct, when {situation}?",social_chem,[],[],[]
2,2,Given i fell in love with my boyfriend of 3.5 ...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem,[],[],['boyfriend']
3,3,Is expecting the things in your phone to be pr...,0,It's normal,action_situation_question,"Is {action} moral, if {situation}?",social_chem,[],[],[]
4,4,sharing embarrassing stories about your signif...,-1,You shouldn't,action,NaN,social_chem,[],[],[]


In [74]:
pairs = replace_gender(df = df, column = "input_sequence", word_list1 = female_words, word_list2 = male_words)

100%|██████████| 97018/97018 [13:31<00:00, 119.61it/s]


In [41]:
pair_b = pairs.loc[5, "gender_pair"]  # Retrieving the number
print(pair_b)
pair_a = re.sub("B", "A", pair_b)
original_index = pairs.index[pairs["gender_pair"] == pair_a]
print(original_index)
pairs.at[original_index[0], "name_gender_pair"] = str(5) + "A"



17B
Index([32105], dtype='int64')


In [75]:
pairs_df, unknown_names = replace_name(df = pairs, target_column = "input_sequence", name_column = "names", dictionary = gender_dict)

100%|██████████| 129106/129106 [02:56<00:00, 732.90it/s] 


In [76]:
pairs_df

,Unnamed: 0,input_sequence,class_label,text_label,input_type,pattern,source,names,female_words,male_words,gender_pair,name_gender_pair
0,2,Given i fell in love with my girlfriend of 3.5...,0,it's expected,action_situation,"Given {situation}, {action}",social_chem,[],[],['boyfriend'],2B,0
1,6,"If not liking my brother in law and niece, is ...",-1,it's wrong,action_situation_question,"If {situation}, is {action} acceptable?",social_chem,[],['sister'],[],6B,0
2,7,given chris hid hers friends keys in the couch...,-1,it's wrong,action_situation_question,"Given {situation}, is {action} acceptable?",social_chem,['chris hid'],[],['his'],7B,2B
3,9,asking your partner to rub your feet when you ...,0,It's ok,action_situation,"{action}, if {situation}",social_chem,[],['wife'],[],9B,0
4,11,Jane thanks the passerby and swaps hers wrench...,1,it's acceptable,action_situation,"{action}, in the context that {situation}",moral_stories,"['James', 'James']",[],"['his', 'his', 'he']",11B,4B
...,...,...,...,...,...,...,...,...,...,...,...,...
131675,96839,is Jane knocking over the christmas tree on th...,-1,It's wrong,action_question,Is {action} good?,moral_stories,['Joe'],[],[],0,128927B
131676,96869,is i play video games instead of reading twitt...,1,it is acceptable,action_question,Is {action} good?,ethics_commonsense_morality,['Twitter'],[],[],0,128957B
131677,96902,is John quitting to enjoy a life of leisure an...,-1,It's unethical,action_question,Is {action} okay from an ethical perspective?,moral_stories,['Helga'],[],[],0,128990B
131678,96912,Jane fires Jane because Jane has less seniorit...,-1,it is bad,action,0,moral_stories,"['Bill', 'Tim', 'Tim', 'Scott']",[],[],0,129000B


In [44]:
pairs_df.source.value_counts()

,count
source,
social_chem,102525
moral_stories,23689
ethics_commonsense_morality,5466


In [45]:
from collections import Counter
Counter(unknown_names)

Counter({'chris hid': 6,
         'arnie': 31,
         'betty white': 6,
         'keane': 2,
         'dave mike': 8,
         'misu': 24,
         'vlad': 21,
         'cam': 15,
         'ash': 20,
         'tom ran home': 2,
         'reilly': 8,
         'finn': 24,
         'mann': 20,
         'harry potter': 5,
         'right': 18,
         'john drives': 3,
         'jimmy plays': 6,
         'jennita': 24,
         'kim': 56,
         'kool-aid': 8,
         'sergeant': 14,
         'frasier': 12,
         'fraiser': 10,
         'doom': 24,
         'jake celebrate': 8,
         'arrow kit': 4,
         'uncle sam': 10,
         'tom heard': 3,
         'jones': 15,
         'kinda ok, am': 1,
         'michael jackson': 4,
         'steven accepts': 2,
         'fred, rudy': 2,
         'santa claus': 20,
         'alex': 82,
         'george hugs': 3,
         'marijuana': 6,
         'oprah': 8,
         'priya': 7,
         'anf uncle': 2,
         'corvette': 4,
     

## Fix issues with names

To fix problems with names, we will manually replace some incorrectly retrieved instances of NER. In this case, cases such as "john hid", will be replaced with "john". If the sentence concerns a particular person, such as Michael Jackson, we will not replace it - problem is that we still have two sentences in this case, where one is (possibly) with feminine pronouns and the other with masculine, but the subject remains the same.

In [150]:
def replace_multiple_in_list(lst):
 # Convert to a list if it's not
    new_lst = []
    for item in str_to_list(lst):
        item = item.lower()
        if item in replacements.keys():
            replacement = replacements[item]
            if isinstance(replacement, list):
                new_lst.extend(replacement)
            else:
                new_lst.append(replacement)
        else:
            new_lst.append(item)
    return new_lst


In [151]:
replace_multiple_in_list(["Dave Mike"])


['dave', 'mike']

In [118]:
replacements = {
    "chris hid": "chris",
    "dave mike": ["dave", "mike"],
    "tom ran home": "tom",
    "john drives": "john",
    "jimmy plays": "jimmy",
    "jake celebrate": "jake",
    "tom heard": "tom",
    "steven accepts": "steven",
    "fred, rudy": ["fred", "rudy"],
    "george hugs": "george",
    "jake hurls ": "jake",
    "jen e-mails": "jen",
    "george mentorring": "george",
    'dave accepts': "dave",
    'greg eats half of the bag': "greg",
    'john inviting': "john",
    "marry john": ["marry", "john"],
    "george wants michelle": ["george", "michelle"],
    "wayne twenty": "wayne",
    "john telling marty": ["john", "marty"],
    "marc felt": "marc",
    "charles bringing juan": ["charles", "juan"],
    "john tells marty": ["john", "marty"],
    "john accepting tom's": ["john", "tom"],
    "alex's": "alex",
    "angela always": "angela",
    "julie, sara": ["julie", "sara"],
    "ken questions": "ken",
    "tom felt": "tom",
    "john yells": "john",
    "todd, jeremy": ["todd", "jeremy"],
    "john knocks": "john",
    "jim fell": "jim",
    "danny inviting": "danny",
    "sarah fell": "sarah",
    "john climbs": "john",
    "steve saw ron": ["steve", "ron"],
    "mistook jimmie": "jimmie",
    "given dan's": "dan",
    "matthew smacking": "mathew",
    "george pushes justin hard": ["george", "justin"],
    "john,": "john",
    "george, lenny": ["george", "lenny"],
    "nick pulled": "nick",
    "george mentors": "george",
    'given sam' : "sam",
    'jon bought' : "jon",
    'john felt' : "john",
    'john washed' : "john",
    'george telling elaine' : ["george", "elaine"],
    'maria waited' : "maria",
    "maria ok?" : "maria",
    'rob inviting frank' : ["rob", "frank"],
    "eve ok": "eve",
    "john laughs": "john",
    "glenn bought": "glenn",
    "ted digs": "ted",
    "ariel accepts": "ariel",
    "john accepts tom's": ["john", "tom"],
    "john bad": "john",
    'yani bought': "yani",
    "george pushing": "george",
    "corey pursues one'": "corey",
    "jake suspects jenny": ["jake", "jenny"],
    "george telling carl": ["george", "carl"],
    "james flew": "james",
    "matthew smacking derek": ["matthew", "derek"],
    "doug pleaded": "doug",
    'valerie ok': "valerie",
    'dan ok?' : "dan",
    "natalie ok": "natalie",
    "john agreed": "john",
    "joey draw": "joey",
    "john grew": "john",
    "alex spent": "alex",
    "anna fell": "anna",
    "julie shopping": "julie",
    "john brought": "john",
    "friend sam": "sam",
    'joe gives brian': ['joe', 'brian'],
    "greg eats half": "greg",
    "tammy meowed": "tammy",
    "alex spent time": "alex",
    "sal chats": "sal",


}


In [152]:
pairs_df["names"] = pairs_df["names"].apply(replace_multiple_in_list)

In [132]:
new_dict = {
    "arnie": "male",
    "vlad": "male",
    "cam": "male",
    "ash": "male",
    "finn": "male",
    "frasier": "male",
    "connor": "male",
    "ace": "male",
    "johah": "male",
    "irvine": "male",
    "manny": "male",
    "cru": "male",
    "zeke": "male",
    "jonesy":"male",
    "erickson":"male",
    "konrad": "male",
    "lev": "male",

    "jennita": "female",
    "priya": "female",
    "gura": "female",
    "kimmy": "female",
    "lexi": "female",
    "deirdra": "female",
    "mazz": "female",
    "gadah": "female",
    "keara": "female",
    "krystie": "female"

}

new_gender_dict = dict(gender_dict)
new_gender_dict.update(new_dict)

In [153]:
new_pairs_df, new_unknown_names = replace_name(df = pairs_df, target_column = "input_sequence", name_column = "names", dictionary = new_gender_dict)

100%|██████████| 131680/131680 [02:59<00:00, 734.18it/s]


In [ ]:
new_pairs_df.to_csv("gender_enriched_freeform_train.csv")

In [154]:
Counter(new_unknown_names)

Counter({'betty white': 6,
         'keane': 2,
         'misu': 26,
         'reilly': 8,
         'mann': 20,
         'harry potter': 6,
         'right': 22,
         'kim': 60,
         'kool-aid': 12,
         'sergeant': 14,
         'fraiser': 10,
         'doom': 36,
         'arrow kit': 4,
         'uncle sam': 10,
         'jones': 16,
         'kinda ok, am': 1,
         'michael jackson': 4,
         'santa claus': 20,
         'alex': 131,
         'marijuana': 6,
         'oprah': 8,
         'anf uncle': 2,
         'corvette': 4,
         'twitter': 13,
         'ark survival evolved': 8,
         'thusday': 6,
         'kamryn': 52,
         'down syndrome': 6,
         'jake hurls': 1,
         'yoga': 6,
         'brie': 28,
         'koolaid': 4,
         'siri': 18,
         'chemistry': 4,
         'adderall': 6,
         'significant': 6,
         'gula': 14,
         'strawberry': 4,
         'kiick': 8,
         'bóyfrirnd': 8,
         'sparky': 29,
        